## Demo : Inception

##### Create level1 vcluster

In [1]:
! vcluster create vc-level1 -n level1 --expose

[info]   Creating namespace level1
[info]   execute command: helm upgrade vc-level1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/141001487 --namespace level1 --install --repository-config='' --values /tmp/3563809011
[done] √ Successfully created virtual cluster vc-level1 in namespace level1. 
- Use 'vcluster connect vc-level1 --namespace level1' to access the virtual cluster
- Use `vcluster connect vc-level1 --namespace level1 -- kubectl get ns` to run a command directly within the vcluster


##### Accessing Level1 Vcluster and Creating a workload

In [2]:
! vcluster connect vc-level1 --namespace level1 --kube-config vc-level1-kubeconfig.yaml

[info]   Waiting for vcluster to come up...
[info]   Using vcluster vc-level1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc-level1 -n level1 -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: vc-level1-kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig vc-level1-kubeconfig.yaml get namespaces`


In [3]:
! kubectl create ns ns1 --kubeconfig ./vc-level1-kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./vc-level1-kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


##### Create Level2 Vcluster

In [4]:
%%bash
export KUBECONFIG=./vc-level1-kubeconfig.yaml 
vcluster create vc-level2 -n level2 --expose

[info]   Creating namespace level2
[info]   execute command: helm upgrade vc-level2 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/3554895994 --namespace level2 --install --repository-config='' --values /tmp/3320340920
[done] √ Successfully created virtual cluster vc-level2 in namespace level2. 
- Use 'vcluster connect vc-level2 --namespace level2' to access the virtual cluster
- Use `vcluster connect vc-level2 --namespace level2 -- kubectl get ns` to run a command directly within the vcluster


##### Accessing Level2 Vcluster and Creating a workload

In [5]:
%%bash
export KUBECONFIG=./vc-level1-kubeconfig.yaml 
vcluster connect vc-level2 --namespace level2 \
    --kube-config vc-level2-kubeconfig.yaml

[info]   Waiting for vcluster to come up...
[info]   Using vcluster vc-level2 load balancer endpoint: 172.18.255.202
[info]   Use `vcluster connect vc-level2 -n level2 -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: vc-level2-kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig vc-level2-kubeconfig.yaml get namespaces`


In [6]:
! kubectl create ns ns1 --kubeconfig ./vc-level2-kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./vc-level2-kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


##### Let's see the Inception from the host cluster

In [8]:
! kubectl get pods -n level1 | grep -e NAME -e sample

NAME                                                              READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-cvrvv-x-ns1-x-vc-level1                         1/1     Running   0          2m31s
sample-7f75ccc884-hzkgj-x-ns1-x-vc-level2-x-level2-x-vc-level1    1/1     Running   0          38s
sample-7f75ccc884-kj5lw-x-ns1-x-vc-level1                         1/1     Running   0          2m31s
sample-7f75ccc884-mx9zm-x-ns1-x-vc-level2-x-level2-x-vc-level1    1/1     Running   0          38s


##### 

### Cleanup

In [9]:
! vcluster delete vc-level1 -n level1
! kubectl delete namespace level1

[info]   Delete helm chart with helm delete vc-level1 --namespace level1 --kubeconfig /tmp/3104529086 --repository-config=''
[done] √ Successfully deleted virtual cluster vc-level1 in namespace level1
[done] √ Successfully deleted virtual cluster pvc data-vc-level1-0 in namespace level1
namespace "level1" deleted


In [10]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   25m
kube-node-lease      Active   25m
kube-public          Active   25m
kube-system          Active   25m
local-path-storage   Active   24m
metallb-system       Active   24m
